## 提取NS feature

In [1]:
import os 
from tqdm import tqdm

### Model 实例化

In [11]:
import torch
import torch.nn as nn
# 计算无关特征网络
class unrelated_conv(nn.Module):
    def __init__(self):
        super(unrelated_conv, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 2048, out_channels= 512, kernel_size=1)
        self.relu_1 = nn.ReLU(True)
        self.maxpool_1 = nn.MaxPool1d(kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=512,out_channels=128,kernel_size=1)
        self.relu_2 = nn.ReLU(True)
        self.maxpool_2 = nn.MaxPool1d(kernel_size=1)

        self.conv3 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=1)
        self.relu_3 = nn.ReLU(True)
        self.maxpool_3 = nn.MaxPool1d(kernel_size=1)

    def forward(self,X):
        X_dsn = X
        temp1_conv1 = self.conv1(X_dsn)
        temp1_relu = self.relu_1(temp1_conv1)
        temp1_maxpool = self.maxpool_1(temp1_relu)
        temp2_conv2 = self.conv2(temp1_maxpool)
        temp2_relu = self.relu_2(temp2_conv2)
        rough_result= self.maxpool_2(temp2_relu)

        rough_result = self.conv3(rough_result)
        rough_result = self.relu_3(rough_result)
        rough_result = self.maxpool_3(rough_result)




        return rough_result

# 计算精细特征网络
class related_conv(nn.Module):
    def __init__(self):
        super(related_conv, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=2048, out_channels=512, kernel_size=1)
        self.relu_1 = nn.ReLU(True)
        self.maxpool_1 = nn.MaxPool1d(kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=512, out_channels=128, kernel_size=1)
        self.relu_2 = nn.ReLU(True)
        self.maxpool_2 = nn.MaxPool1d(kernel_size=1)

        self.conv3 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=1)
        self.relu_3 = nn.ReLU(True)
        self.maxpool_3 = nn.MaxPool1d(kernel_size=1)



    def forward(self,X):
        X_dsn = X
        temp1_conv1 = self.conv1(X_dsn)
        temp1_relu = self.relu_1(temp1_conv1)
        temp1_maxpool = self.maxpool_1(temp1_relu)
        temp2_conv2 = self.conv2(temp1_maxpool)
        temp2_relu = self.relu_2(temp2_conv2)
        smooth_result = self.maxpool_2(temp2_relu)

        smooth_result = self.conv3(smooth_result)
        smooth_result = self.relu_3(smooth_result)
        smooth_result = self.maxpool_3(smooth_result)


        return smooth_result
#输入数据为精细特征，预测结果
class predict_part(nn.Module):
    def __init__(self):
        super(predict_part, self).__init__()
        self.fc1 = nn.Linear(in_features=32,out_features=1)
        self.relu = nn.ReLU(True)
        self.fc2 = nn.Linear(in_features=64,out_features=1)
    def forward(self,x):
        x_in = x.reshape(x.shape[0],-1)
        x_in = self.relu(x_in)
        predict_temp1 = self.fc1(x_in)
        # predict_temp2 = self.relu(predict_temp1)
        # predict_result = self.relu(self.fc2(predict_temp2))

        return predict_temp1
#特征复原。。
class encoder_image(nn.Module):
    def __init__(self):
        super(encoder_image, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=32, out_channels=128, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=512, kernel_size=1)

        self.conv3 = nn.Conv1d(in_channels=512, out_channels=2048, kernel_size=1)

    def forward(self, smooth_feature, rough_feature):
        encode_feature = torch.add(smooth_feature, rough_feature)
        encode_1 = self.conv1(encode_feature)
        x_hat = self.conv2(encode_1)

        x_hat = self.conv3(x_hat)
        return x_hat



class DSN(nn.Module):
    def __init__(self):
        super(DSN, self).__init__()
        self.unrelated_conv = unrelated_conv()
        self.related_conv = related_conv()
        self.predict_part = predict_part()
        self.encoder = encoder_image()

    def forward(self,X_feature):

        unrealted_data = self.unrelated_conv(X_feature)


        realted_data = self.related_conv(X_feature)


        predict_result  = self.predict_part(realted_data)


        encode_result = self.encoder(unrealted_data, realted_data)

        return unrealted_data, realted_data, predict_result, encode_result

# model = DSN()
# x = torch.rand([600,1,2048])
# x = x.permute(0,2,1)
# print(f'x:{x.shape}')
# unrealted_data, realted_data, predict_result, encode_result = model(x)
# print(f'unrealted_data:{unrealted_data.shape}')
# print(f'predict_Result :{predict_result.shape}')
# print(f'related_data:{realted_data.shape}')
# print(f'encode_result:{encode_result.shape}')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataloader读取

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
import os
from os.path import join as pjoin
import scipy.io as scio


class DSN_Dataset(nn.Module):
    def __init__(self, file_path, file_index):
        super(DSN_Dataset, self).__init__()
        self.file_index = np.load(file_index).tolist()
        self.file_path = file_path
        self.mat_name = [str(i) + '.mat' for i in self.file_index]  ## 读取对应id，来区分训练&验证，测试数据集
        self.mat_path = [pjoin(self.file_path, i) for i in self.mat_name]

    def __getitem__(self, index):
        mat_file = self.mat_path[index]
        
        mat_name = mat_file.split('/')[-1].rstrip('.mat')

        mat_data = scio.loadmat(mat_file)
        

        feature_np = mat_data['feature']

        label_np = mat_data['check_label']

        feature_th = torch.from_numpy(feature_np)

        label_th = torch.from_numpy(label_np)

        return feature_th, label_th, mat_name

    def __len__(self):
        return len(self.mat_path)


# Train_Dsn_dataset = DSN_Dataset(file_path='/hy-tmp/Feature_fusion_mta_save/',
#                                 file_index='/hy-tmp/train_valid_index.npy')

# Dsn_dataloader = DataLoader(Train_Dsn_dataset, shuffle=True, batch_size=1)

# for data in Dsn_dataloader:
#     feature, label,_ = data  ## feature: [1,rand,2048]  label:[1,rand,1]

#     feature = feature.squeeze(dim=0)  ## [rand,2048]

#     feature = feature.unsqueeze(dim=1)  ## [rand, 1, 2048]

#     feature = feature.permute(0, 2, 1)  ## [rand, 2048 ,1 ]

#     label = label.squeeze(dim=0)  ## [rand,1]

#     print(f'feature:{feature.shape}')  ## check. :: [rand,2048,1]

#     print(f'label:{label.shape}')  ## check :: [rand,1]

#     break


### 读取实验数据

In [32]:
### txt 数据读取 用作后续的数据集准备
file_path = '/hy-tmp/Feature_fusion_resnet_mta_save/'
train_file_index = '/hy-tmp/train_valid_index.npy'
test_file_index = '/hy-tmp/test_index.npy'

### dataset 准备
#### train/valid/test -> batch size
train_batch_size = 1
test_batch_size = 1

#### train dataloader


train_dataset = DSN_Dataset(file_path=file_path,file_index=train_file_index)

train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1)

### test dataloader
test_dataset = DSN_Dataset(file_path=file_path,file_index=test_file_index)

test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

### model 实例化

#### 模式实例化参数

model = DSN()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.load_state_dict(torch.load('/hy-tmp/Code/Best_weights/Resnet-NS/Epcoh_1049_Rmse_6.62206506729126_PCC_0.2721236440806091_CCC_0.16563803434174354.pth'))
model.eval()


model = model.to(device)

## 保存路径

save_dir = '/hy-tmp/Feature_fusion_resnet_ns_save/'

save_dir_check = os.path.exists(save_dir)

if not save_dir_check:
        os.mkdir(save_dir)


for valid_data in tqdm(test_loader):
    
    feature, label,mat_name = valid_data  ## feature: [1,rand,2048]  label:[1,rand,1]

    feature = feature.squeeze(dim=0)  ## [rand,2048]

    feature = feature.unsqueeze(dim=1)  ## [rand, 1, 2048]

    feature = feature.permute(0, 2, 1)  ## [rand, 2048 ,1 ]

    label = label.squeeze(dim=0)  ## [rand,1]

    feature = feature.to(device)
    label = label.to(device)

    unrealted_data, related_data, predict_result, encode_result = model(feature)

    related_data = related_data.to('cpu').data.squeeze(dim=2).numpy()

    label = label.to('cpu').data.squeeze(dim=1).numpy()
     
    
    save_mat_file = save_dir + mat_name[0] + '.mat'
    
    
    scio.savemat(save_mat_file,{'feature':related_data, 'label': label})


100%|██████████| 56/56 [00:00<00:00, 104.12it/s]


In [34]:
print(len(os.listdir('/hy-tmp/Feature_fusion_resnet_mta_save/')))

275


In [2]:
import  scipy.io as scio

In [5]:
scio.loadmat('/hy-tmp/Feature_fusion_resnet_ns_save/300.mat')['feature'].shape

(648, 32)

In [8]:
data_dir = '/hy-tmp/Feature_fusion_resnet_ns_save/'

mat_file = [data_dir[8:] + i for i in os.listdir(data_dir)]


In [ ]:
mat_file

['Feature_fusion_resnet_ns_save/300.mat',
 'Feature_fusion_resnet_ns_save/301.mat',
 'Feature_fusion_resnet_ns_save/302.mat',
 'Feature_fusion_resnet_ns_save/303.mat',
 'Feature_fusion_resnet_ns_save/304.mat',
 'Feature_fusion_resnet_ns_save/305.mat',
 'Feature_fusion_resnet_ns_save/306.mat',
 'Feature_fusion_resnet_ns_save/307.mat',
 'Feature_fusion_resnet_ns_save/308.mat',
 'Feature_fusion_resnet_ns_save/309.mat',
 'Feature_fusion_resnet_ns_save/310.mat',
 'Feature_fusion_resnet_ns_save/311.mat',
 'Feature_fusion_resnet_ns_save/312.mat',
 'Feature_fusion_resnet_ns_save/313.mat',
 'Feature_fusion_resnet_ns_save/314.mat',
 'Feature_fusion_resnet_ns_save/315.mat',
 'Feature_fusion_resnet_ns_save/316.mat',
 'Feature_fusion_resnet_ns_save/317.mat',
 'Feature_fusion_resnet_ns_save/318.mat',
 'Feature_fusion_resnet_ns_save/319.mat',
 'Feature_fusion_resnet_ns_save/320.mat',
 'Feature_fusion_resnet_ns_save/321.mat',
 'Feature_fusion_resnet_ns_save/322.mat',
 'Feature_fusion_resnet_ns_save/32

In [17]:
import numpy as np
import pandas as pd
def concordance_correlation_coefficient(y_true, y_pred):
    """Concordance correlation coefficient."""
    # Remove NaNs
    df = pd.DataFrame({
        'y_true': y_true,
        'y_pred': y_pred
    })
    df = df.dropna()
    y_true = df['y_true']
    y_pred = df['y_pred']
    # Pearson product-moment correlation coefficients
    cor = np.corrcoef(y_true, y_pred)[0][1]
    # Mean
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    # Variance
    var_true = np.var(y_true)
    var_pred = np.var(y_pred)
    # Standard deviation
    sd_true = np.std(y_true)
    sd_pred = np.std(y_pred)
    # Calculate CCC
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    return numerator / denominator




In [29]:

from scipy.stats import pearsonr
### txt 数据读取 用作后续的数据集准备
file_path = '/hy-tmp/Feature_fusion_resnet_mta_save/'
train_file_index = '/hy-tmp/train_valid_index.npy'
test_file_index = '/hy-tmp/test_index.npy'

### dataset 准备
#### train/valid/test -> batch size
train_batch_size = 1
test_batch_size = 1

#### train dataloader


train_dataset = DSN_Dataset(file_path=file_path,file_index=train_file_index)

train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1)

### test dataloader
test_dataset = DSN_Dataset(file_path=file_path,file_index=test_file_index)

test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

### model 实例化

#### 模式实例化参数

model = DSN()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

path_dir = '/hy-tmp/Code/DSN_results_v2/2022-09-03-13:54:04/weights/'
model_path = [path_dir + i for i in os.listdir(path_dir)]
for i in model_path:
    model.load_state_dict(torch.load(i))
    model.eval()
    loss_1 = nn.MSELoss()

    model = model.to(device)

    epoch = 0

    ## 保存路径

    save_dir = '/hy-tmp/Feature_fusion_resnet_ns_save/'

    save_dir_check = os.path.exists(save_dir)

    if not save_dir_check:
            os.mkdir(save_dir)

    total_valid_labels = torch.Tensor()

    total_valid_predicts = torch.Tensor()

    for valid_data in tqdm(test_loader):
        feature, label,mat_name = valid_data  ## feature: [1,rand,2048]  label:[1,rand,1]

        feature = feature.squeeze(dim=0)  ## [rand,2048]

        feature = feature.unsqueeze(dim=1)  ## [rand, 1, 2048]

        feature = feature.permute(0, 2, 1)  ## [rand, 2048 ,1 ]

        label = label.squeeze(dim=0)  ## [rand,1]

        feature = feature.to(device)
        label = label.to(device)

        unrealted_data, realted_data, predict_result, encode_result = model(feature)

        predict_result = predict_result.to('cpu').data

        label = label.to('cpu').data
        ##  predict  result
        total_valid_predicts = torch.cat((total_valid_predicts, predict_result), 0)
        ## ground truthg
        total_valid_labels = torch.cat((total_valid_labels, label), 0)


    ###  计算评估指标
    Metric_rmse = loss_1(total_valid_predicts, total_valid_labels).pow(0.5).item()
    Metric_pcc = pearsonr(total_valid_predicts.reshape(-1), total_valid_labels.reshape(-1))[0]
    Metric_ccc = concordance_correlation_coefficient(total_valid_predicts.reshape(-1), total_valid_labels.reshape(-1))
    if Metric_rmse<6.8 or Metric_ccc>0.15:
        print(f"Epcoh:{epoch}:Rmse{Metric_rmse},Pcc:{Metric_pcc},CCC:{Metric_ccc}")
        print(i)


100%|██████████| 56/56 [00:01<00:00, 50.78it/s]


Epcoh:0:Rmse7.092784404754639,Pcc:0.23982270850563506,CCC:0.1516453884441325
/hy-tmp/Code/DSN_results_v2/2022-09-03-13:54:04/weights/Epcoh:29_Rmse:7.092784404754639_PCC:0.23982270850563506_CCC:0.1516453884441325.pth


100%|██████████| 56/56 [00:01<00:00, 50.57it/s]


Epcoh:0:Rmse7.185478210449219,Pcc:0.2457333085238638,CCC:0.15189431826348077
/hy-tmp/Code/DSN_results_v2/2022-09-03-13:54:04/weights/Epcoh:38_Rmse:7.185478210449219_PCC:0.2457333085238638_CCC:0.15189431826348077.pth


100%|██████████| 56/56 [00:01<00:00, 53.60it/s]


...

Epcoh:0:Rmse7.0807671546936035,Pcc:0.2526166404763505,CCC:0.16346588016198182
/hy-tmp/Code/DSN_results_v2/2022-09-03-13:54:04/weights/Epcoh:74_Rmse:7.0807671546936035_PCC:0.2526166404763505_CCC:0.16346588016198182.pth


100%|██████████| 56/56 [00:01<00:00, 50.04it/s]


Epcoh:0:Rmse7.028863906860352,Pcc:0.2530509141042567,CCC:0.16463282490647396
/hy-tmp/Code/DSN_results_v2/2022-09-03-13:54:04/weights/Epcoh:80_Rmse:7.028863906860352_PCC:0.2530509141042567_CCC:0.16463282490647396.pth


100%|██████████| 56/56 [00:01<00:00, 53.35it/s]

Epcoh:0:Rmse7.05543327331543,Pcc:0.2557936359392331,CCC:0.16571836347933616
/hy-tmp/Code/DSN_results_v2/2022-09-03-13:54:04/weights/Epcoh:82_Rmse:7.05543327331543_PCC:0.2557936359392331_CCC:0.16571836347933616.pth


In [32]:

from scipy.stats import pearsonr
### txt 数据读取 用作后续的数据集准备
file_path = '/hy-tmp/Feature_fusion_resnet_mta_save/'
train_file_index = '/hy-tmp/train_valid_index.npy'
test_file_index = '/hy-tmp/test_index.npy'

### dataset 准备
#### train/valid/test -> batch size
train_batch_size = 1
test_batch_size = 1

#### train dataloader


train_dataset = DSN_Dataset(file_path=file_path,file_index=train_file_index)

train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1)

### test dataloader
test_dataset = DSN_Dataset(file_path=file_path,file_index=test_file_index)

test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

### model 实例化

#### 模式实例化参数

model = DSN()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# path_dir = '/hy-tmp/Code/DSN_results_v2/2022-09-03-13:54:04/weights/'
# model_path = [path_dir + i for i in os.listdir(path_dir)]

model_path = ['/hy-tmp/Code/Best_weights/Resnet-NS/Epcoh_1049_Rmse_6.62206506729126_PCC_0.2721236440806091_CCC_0.16563803434174354.pth']
for i in model_path:
    model.load_state_dict(torch.load(i))
    model.eval()
    loss_1 = nn.MSELoss()

    model = model.to(device)

    epoch = 0

    ## 保存路径

    save_dir = '/hy-tmp/Feature_fusion_resnet_ns_save/'

    save_dir_check = os.path.exists(save_dir)

    if not save_dir_check:
            os.mkdir(save_dir)

    total_valid_labels = torch.Tensor()

    total_valid_predicts = torch.Tensor()

    for valid_data in tqdm(test_loader):
        feature, label,mat_name = valid_data  ## feature: [1,rand,2048]  label:[1,rand,1]

        feature = feature.squeeze(dim=0)  ## [rand,2048]

        feature = feature.unsqueeze(dim=1)  ## [rand, 1, 2048]

        feature = feature.permute(0, 2, 1)  ## [rand, 2048 ,1 ]

        label = label.squeeze(dim=0)  ## [rand,1]

        feature = feature.to(device)
        label = label.to(device)

        unrealted_data, realted_data, predict_result, encode_result = model(feature)

        predict_result = predict_result.to('cpu').data

        label = label.to('cpu').data
        ##  predict  result
        total_valid_predicts = torch.cat((total_valid_predicts, predict_result), 0)
        ## ground truthg
        total_valid_labels = torch.cat((total_valid_labels, label), 0)


    ###  计算评估指标
    
    print(total_valid_predicts.shape)
    Metric_rmse = loss_1(total_valid_predicts, total_valid_labels).pow(0.5).item()
    Metric_pcc = pearsonr(total_valid_predicts.reshape(-1), total_valid_labels.reshape(-1))[0]
    Metric_ccc = concordance_correlation_coefficient(total_valid_predicts.reshape(-1), total_valid_labels.reshape(-1))
    if Metric_rmse<6.8 or Metric_ccc>0.15:
        print(f"Epcoh:{epoch}:Rmse{Metric_rmse},Pcc:{Metric_pcc},CCC:{Metric_ccc}")
        print(i)


100%|██████████| 56/56 [00:01<00:00, 42.81it/s]

torch.Size([54319, 1])
Epcoh:0:Rmse6.874358177185059,Pcc:0.2526884529692214,CCC:0.1931105764395173
/hy-tmp/Code/Best_weights/Resnet-NS/Epcoh_1049_Rmse_6.62206506729126_PCC_0.2721236440806091_CCC_0.16563803434174354.pth
